In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import sqlite3 as lite
import pandas as pd

In [3]:
date = '2019-01-19'
hot_url = 'https://www.billboard.com/charts/hot-100/'
hot_html = requests.get(hot_url + date).text
hot_soup = BeautifulSoup(hot_html, 'lxml')

In [13]:
# for the first ranked song
rank = '1'
title = hot_soup.find('div', attrs={'class':'chart-number-one__title'}).text
artist = hot_soup.find('div', attrs={'class': 'chart-number-one__artist'}).text.strip('\n')

print(rank, title, artist, sep=",")

1,Sunflower (Spider-Man: Into The Spider-Verse),Post Malone & Swae Lee


In [20]:
# for the rest
chart_list = hot_soup.find_all('div', attrs={'class': 'chart-list-item'})

rank = chart_list[0].attrs['data-rank'] 
title = chart_list[0].attrs['data-title']
artist = chart_list[0].attrs['data-artist']

In [21]:
def hot_scraper(year, month):
    date = str(year) + '-' + month
    hot_url = hot_url = 'https://www.billboard.com/charts/hot-100/'
    hot_html = requests.get(hot_url + date + '-15').text
    hot_soup = BeautifulSoup(hot_html, 'lxml')
    
    result_list = []
    #for the chart number one
    rank = '1'
    title = hot_soup.find('div', attrs={'class':'chart-number-one__title'}).text
    artist = hot_soup.find('div', attrs={'class': 'chart-number-one__artist'}).text.strip('\n')
    result_list.append((date, rank, title, artist))
    
    #for the rest
    chart_list = hot_soup.find_all('div', attrs={'class': 'chart-list-item'})
    for chart in chart_list:
        rank = chart.attrs['data-rank'] 
        title = chart.attrs['data-title']
        artist = chart.attrs['data-artist']
        
        result_list.append((date, rank, title, artist))
        
    return result_list

In [22]:
def get_all_hot():
    month = '01'
    month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
    hot_list = []
    
    for year in range(2017, 2018+1):
        for month in month_list:
            hot_list.extend(hot_scraper(year, month))
            
    return hot_list

In [24]:
result_list = get_all_hot()

In [25]:
db = lite.connect('./chart_research.db')

In [26]:
pd.options.display.max_colwidth = 300

In [27]:
len(result_list)

2400

In [28]:
result_list[0]

('2017-01', '1', 'Bad And Boujee', 'Migos Featuring Lil Uzi Vert')

In [30]:
query_create_table = """
CREATE TABLE IF NOT EXISTS
Hot100
("Date" TEXT,
"Rank" TEXT,
"Title" TEXT,
"Artist" TEXT);
"""

query_insert_data = """
INSERT INTO Hot100 VALUES(?,?,?,?);
"""

with db:
    cur = db.cursor()
    cur.execute(query_create_table)
    for row in result_list:
        cur.execute(query_insert_data, row)

In [31]:
# for check
query = """
SELECT * FROM Hot100; 
"""

pd.read_sql(query, db).head()

,Date,Rank,Title,Artist
0,2017-01,1,Bad And Boujee,Migos Featuring Lil Uzi Vert
1,2017-01,2,Black Beatles,Rae Sremmurd Featuring Gucci Mane
2,2017-01,3,Closer,The Chainsmokers Featuring Halsey
3,2017-01,4,Starboy,The Weeknd Featuring Daft Punk
4,2017-01,5,24K Magic,Bruno Mars
